In [1]:
import requests
from selenium import webdriver
from splinter import Browser
from bs4 import BeautifulSoup
import shutil
import tweepy
from IPython.display import Image
import pandas as pd 
import time 
from flask import Flask


In [2]:
driver = webdriver.Chrome(r'C:\\Users\\rohit\\Anaconda3.3\\Lib\site-packages\\chromedriver\\chromedriver.exe')
#executable_path = {"executable_path": driver}
executable_path = {"executable_path": r"C:\\Users\\rohit\\Anaconda3.3\\Lib\site-packages\\chromedriver\\chromedriver.exe"}


In [3]:
browser = Browser("chrome", **executable_path, headless=False)

#req = requests.get('https://mars.nasa.gov/news/')
#soup = BeautifulSoup(req.text, "lxml")

url = 'https://mars.nasa.gov/news/'
browser.visit(url)

html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [4]:
article = soup.find("div", class_="list_text")
news_p = article.find("div", class_="article_teaser_body").text
news_title = article.find("div", class_="content_title").text
news_date = article.find("div", class_="list_date").text
print(news_date)
print(news_title)
print(news_p)

May 23, 2018
InSight Steers Toward Mars
The spacecraft has completed its first trajectory correction maneuver.


In [5]:
### JPL Mars Space Images - Featured Image
secURL = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(secURL)

In [6]:
# Scrape the browser into soup and use soup to find the image of mars
# Save the image url to a variable called `img_url`
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
image = soup.find("img", class_="thumb")["src"]

imgURL = "https://jpl.nasa.gov"+image
featuredImgURL = imgURL

In [7]:
# Use the requests library to download and save the image from the `imgURL` above
response = requests.get(imgURL, stream=True)
with open('img.jpg', 'wb') as out_file:
    shutil.copyfileobj(response.raw, out_file)
    

In [8]:
# Display the image with IPython.display
Image(url='img.jpg')

In [11]:
#Visit the Mars Weather twitter account [here](https://twitter.com/marswxreport?lang=en) 
#and scrape the latest Mars weather tweet from the page. Save the tweet text for the weather 
#report as a variable called `mars_weather`.

#Twitter API Keys
from config import consumer_key, consumer_secret, access_token, access_token_secret

#Setup Tweepy API Authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())
target_user = "marswxreport" 
full_tweet = api.user_timeline(target_user , count = 1)
mars_weather=full_tweet[0]['text']
mars_weather

'Sol 2060 (May 23, 2018), Sunny, high 4C/39F, low -73C/-99F, pressure at 7.43 hPa, daylight 05:20-17:20'

In [12]:
#Visit the Mars facts webpage and scrape table data into Pandas
thirdURL = "http://space-facts.com/mars/"
browser.visit(thirdURL)

In [13]:
#Use Pandas to convert the data to a HTML table string.
readData = pd.read_html(thirdURL)
mars_data=pd.DataFrame(readData[0])
mars_data.columns=['Mars','Data']
mars_table=mars_data.set_index("Mars")
marsdata = mars_table.to_html(classes='marsdata')
marsdata=marsdata.replace('\n', ' ')
marsdata

'<table border="1" class="dataframe marsdata">   <thead>     <tr style="text-align: right;">       <th></th>       <th>Data</th>     </tr>     <tr>       <th>Mars</th>       <th></th>     </tr>   </thead>   <tbody>     <tr>       <th>Equatorial Diameter:</th>       <td>6,792 km</td>     </tr>     <tr>       <th>Polar Diameter:</th>       <td>6,752 km</td>     </tr>     <tr>       <th>Mass:</th>       <td>6.42 x 10^23 kg (10.7% Earth)</td>     </tr>     <tr>       <th>Moons:</th>       <td>2 (Phobos &amp; Deimos)</td>     </tr>     <tr>       <th>Orbit Distance:</th>       <td>227,943,824 km (1.52 AU)</td>     </tr>     <tr>       <th>Orbit Period:</th>       <td>687 days (1.9 years)</td>     </tr>     <tr>       <th>Surface Temperature:</th>       <td>-153 to 20 °C</td>     </tr>     <tr>       <th>First Record:</th>       <td>2nd millennium BC</td>     </tr>     <tr>       <th>Recorded By:</th>       <td>Egyptian astronomers</td>     </tr>   </tbody> </table>'

In [14]:
#Mars Hemisperes
#Visit the USGS Astrogeology site to obtain high resolution images for each of Mar's hemispheres.
forthURL = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(forthURL)

In [15]:
#Use splinter to loop through the images and load them into a dictionary
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
mars_hemis=[]

In [ ]:
#find the image url to the full resolution image
#loop through the four tags and load the data to the dictionary
for i in range (4):
    time.sleep(5)
    images = browser.find_by_tag('h3')
    images[i].click()
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    partial = soup.find("img", class_="wide-image")["src"]
    img_url = 'https://astrogeology.usgs.gov'+ partial
    img_title = soup.find("h2",class_="title").text
    dictionary={"title":img_title,"img_url":img_url}
    mars_hemis.append(dictionary)
    browser.back()  
print(mars_hemis)